In [2]:
# Initialisation et chargement des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Chargement du Dataset
df = pd.read_csv(r"C:\Users\fatim\PycharmProjects\ProLib#\House_Pricing\Data\dirty_cafe_sales.csv")

# Afficher les premières lignes (Exigence du PDF)
print("Aperçu du jeu de données :")
print(df.head())

Aperçu du jeu de données (df.head()):
  Transaction ID    Item Quantity Price Per Unit Total Spent  Payment Method  \
0    TXN_1961373  Coffee        2            2.0         4.0     Credit Card   
1    TXN_4977031    Cake        4            3.0        12.0            Cash   
2    TXN_4271903  Cookie        4            1.0       ERROR     Credit Card   
3    TXN_7034554   Salad        2            5.0        10.0         UNKNOWN   
4    TXN_3160411  Coffee        2            2.0         4.0  Digital Wallet   

   Location Transaction Date  
0  Takeaway       2023-09-08  
1  In-store       2023-05-16  
2  In-store       2023-07-19  
3   UNKNOWN       2023-04-27  
4  In-store       2023-06-11  


In [4]:
# Taille et Types de Variables
print("\nTaille du Dataset (lignes, colonnes) :")
print(df.shape)

print("\nInformation sur les types de données et valeurs non-nulles  :")
df.info()



Taille du Dataset (lignes, colonnes) :
(10000, 8)

Information sur les types de données et valeurs non-nulles (df.info()) :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


Le dataset des ventes de café est composé de 10 000 transactions et de 8 colonnes (caractéristiques). L'analyse des types de données et des valeurs non-nulles révèle plusieurs problèmes critiques qui devront être adressés dans la phase de Traitement Avancé.

- Problème Critique : Toutes les Colonnes sont de Type Texte (Object) :
  # Quantity :Le calcul de la quantité totale vendue est impossible.
  # Price Per Unit : Impossible de déterminer le prix moyen ou de détecter les prix aberrants.
  # Total Spent	: La prédiction de la variable cible (revenu) est bloquée car la colonne n'est pas numérique.
  # Transaction Date : Les analyses temporelles (ventes par mois/saison) sont impossibles.
   -------La transformation des types de données sera la première priorité du nettoyage.
-me des Valeurs Manquantes (Non-Null Count) 

In [5]:
# Compte des Valeurs Manquantes
print("\nNombre de valeurs manquantes (NaN) par colonne :")
print(df.isnull().sum())
 
# Les colonnes avec des valeurs manquantes devront être imputées ou supprimées lors du Traitement Avancé.


Nombre de valeurs manquantes (NaN) par colonne :
Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64


L'analyse du nombre de valeurs nulles par colonne confirme la nature de " cafe sales Data" et met en évidence trois catégories de colonnes à traiter lors de la Phase 3 (Traitement Avancé).

Location (32.65% - Élevé) : L'absence de localisation pour près du tiers des ventes est un problème majeur pour l'analyse spatiale et doit être gérée par imputation par le mode (la localisation la plus fréquente) ou en créant une catégorie "Inconnu".

Payment Method	(25.79% - Élevé) :Le mode de paiement est manquant pour plus d'un quart des transactions.la meilleure approche sera d'imputer ces valeurs par une catégorie 'MANQUANT' ou 'INCONNU'.

Item (3.33%	- Modéré) : L'article vendu est inconnu pour 333 transactions.

Quantity, Price Per Unit, Total Spent, Transaction Date	( 138 à 179	≈1.4% à 1.8% -Faible) : Pour les colonnes clés (prix, quantité, date), le taux est gérable. La meilleure pratique sera l'imputation par la médiane pour les valeurs numériques et par le mode pour la date, ou simplement la suppression des lignes concernées si cela n'affecte pas trop l'échantillon.

In [7]:
# Statistiques sur les variables numériques
# Nous utilisons 'include="all"' pour voir aussi les statistiques des variables 'object'
# Ceci permet de voir le 'top' et la 'frequence' des catégories.
print("\nStatistiques Descriptives :")
print(df.describe(include='all'))

# Analyser la distribution des catégories clés
print("\nFréquence des Catégories de Produits :")
print(df['Item'].value_counts())


Statistiques Descriptives :
       Transaction ID   Item Quantity Price Per Unit Total Spent  \
count           10000   9667     9862           9821        9827   
unique          10000     10        7              8          19   
top       TXN_1961373  Juice        5            3.0         6.0   
freq                1   1171     2013           2429         979   

        Payment Method  Location Transaction Date  
count             7421      6735             9841  
unique               5         4              367  
top     Digital Wallet  Takeaway          UNKNOWN  
freq              2291      3022              159  

Fréquence des Catégories de Produits :
Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64


Colonnes numériques inutilisables : Pour les colonnes Quantity, Price Per Unit, et Total Spent, les statistiques clés comme la moyenne (mean), l'écart-type (std), le minimum (min) et le maximum (max) sont absentes.

# Problèmes de Types (Bloquant) : Toutes les colonnes numériques et la colonne de date sont mal lues (object).

# Problèmes de Valeurs Manquantes (Élevé) : Les colonnes Location et Payment Method manquent de 25% à 32% des données.

# Problèmes de Cohérence (Saisie Sale) : Des valeurs textuelles comme ERROR et UNKNOWN sont présentes dans des colonnes qui devraient être numériques ou catégorielles cohérentes.

In [8]:
# Pourcentage de valeurs manquantes
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("Pourcentage de valeurs manquantes par colonne :")
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

Pourcentage de valeurs manquantes par colonne :
Location            32.65
Payment Method      25.79
Item                 3.33
Price Per Unit       1.79
Total Spent          1.73
Transaction Date     1.59
Quantity             1.38
dtype: float64


In [9]:
# Décompte des valeurs uniques dans les colonnes clés
print("\nNombre de valeurs uniques pour les colonnes clés :")
print(df[['Item', 'Transaction ID']].nunique())


Nombre de valeurs uniques pour les colonnes clés :
Item                 10
Transaction ID    10000
dtype: int64


In [11]:
# Vérification des colonnes numériques pour les incohérences
# On vérifie les valeurs uniques des colonnes qui devraient être numériques mais sont 'object'
print("\nExamen des 10 valeurs uniques dans les colonnes 'Price Per Unit', 'Total Spent' et 'Quantity' :")
print(df['Price Per Unit'].value_counts().head(10),'\n')
print(df['Total Spent'].value_counts().head(10),'\n')
print(df['Quantity'].value_counts().head(10),'\n')



Examen des 10 valeurs uniques dans les colonnes 'Price Per Unit', 'Total Spent' et 'Quantity' :
Price Per Unit
3.0        2429
4.0        2331
2.0        1227
5.0        1204
1.0        1143
1.5        1133
ERROR       190
UNKNOWN     164
Name: count, dtype: int64 

Total Spent
6.0     979
12.0    939
3.0     930
4.0     923
20.0    746
15.0    734
8.0     677
10.0    524
2.0     497
9.0     479
Name: count, dtype: int64 

Quantity
5          2013
2          1974
4          1863
3          1849
1          1822
UNKNOWN     171
ERROR       170
Name: count, dtype: int64 



 # La colonne Total Spent est le point le plus faible du dataset et nécessite une résolution urgente.

# Hypothèse : La colonne Total Spent a été mal saisie ou a été encodée de manière erronée avec seulement 19 niveaux de facteurs(les différentes catégories uniques que la variable peut prendre)

# Tâche avancee (Nettoyage) :  la conversion en float64 , le Modélisateur devra s'assurer que ces 19 valeurs uniques ne masquent pas un problème d'échantillonnage ou de formatage non détecté.

In [15]:
# Voir le quantile pour voir la dispersion au-delà de la moyenne
print("\nStatistiques sur le Total des Ventes :")
print(df['Total Spent'].describe(percentiles=[.01, .25, .50, .75, .99]))


Statistiques sur le Total des Ventes :
count     9827
unique      19
top        6.0
freq       979
Name: Total Spent, dtype: object


In [19]:
#  Conversion de la colonne 'date' en format datetime
# Ceci est une étape de "nettoyage", mais essentielle à l'exploration
try:
    df['date'] = pd.to_datetime(df['Transaction Date'])
    print("\nConversion de la colonne 'date' en datetime réussie.")
except Exception as e:
    print(f"\nErreur lors de la conversion de la date : {e}")


Erreur lors de la conversion de la date : time data "ERROR" doesn't match format "%Y-%m-%d", at position 11. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


 # Analyse et Visualisation	Bloquée. il n'est pas possible de créer des graphiques d'évolution des ventes dans le temps (Série Temporelle) tant que la colonne Transaction Date n'est pas nettoyée.
# Traitement Avancé : Nettoyage	Priorité Absolue. nécessaire pour la visualisation  de caractéristiques (créer des variables Mois, Jour de la semaine).

In [20]:
# Période couverte par le dataset
print("\nPériode de temps couverte :")
print(f"Date de la première transaction : {df['Transaction Date'].min()}")
print(f"Date de la dernière transaction : {df['Transaction Date'].max()}")


Période de temps couverte :


TypeError: '<=' not supported between instances of 'str' and 'float'